In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import SQL Alchemy
import sqlalchemy
from sqlalchemy import create_engine, MetaData

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [3]:
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Text, Numeric

from sqlalchemy import inspect

In [4]:
#csv to df
access_logs_df = pd.read_csv('/home/spinster/bootcamp/homework/apache/infiles/cleaned_access_logs.csv')
responses_df = pd.read_csv('/home/spinster/bootcamp/homework/apache/infiles/coordinates.csv')


In [5]:
#check dtypes and NaN
access_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30694 entries, 0 to 30693
Data columns (total 10 columns):
Unnamed: 0      30694 non-null int64
ip              30694 non-null object
DateTime        30694 non-null object
TimeZone        30694 non-null object
Method          30694 non-null object
URL             30694 non-null object
ResponseCode    30694 non-null object
BytesSent       30694 non-null object
Referer         30694 non-null object
UserAgent       30694 non-null object
dtypes: int64(1), object(9)
memory usage: 2.3+ MB


In [6]:
#check dtypes and NaN
responses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7724 entries, 0 to 7723
Data columns (total 4 columns):
Unnamed: 0    7724 non-null int64
ip            7724 non-null object
latitude      7724 non-null float64
longitude     7724 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 241.5+ KB


In [7]:
# create engine
engine = create_engine('sqlite:///apache.sqlite')

In [8]:
# interface with engine via metadata
Base.metadata.create_all(engine)

In [9]:
#start session
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [10]:
#convert dfs to dicts in order to create classes
access = access_logs_df.to_dict(orient='records')
responses = responses_df.to_dict(orient='records')

In [11]:
#create classes
Base = declarative_base()

class Access(Base):
    __tablename__ = 'access'
    
    ip = Column(String, primary_key=True)
    DateTime= Column(String)
    Method = Column(String)
    URL = Column(String)
    ResponseCode = Column(String)
    Referer = Column(String)
    BytesSent = Column(String)
    UserAgent = Column(String)

    def __repr__(self):
        return f"id={self.id}, name={self.access}"

In [12]:
class Responses(Base):
    __tablename__ = 'responses'
    ip = Column(String, primary_key=True)
    latitude = Column(Float)
    longitude = Column(Float)
    def __repr__(self):
        return f"id={self.id}, name={self.responses}"

In [13]:
# create db
Base.metadata.create_all(engine)

In [14]:
# check that it worked.
engine.table_names()

['access', 'responses']

In [15]:
# create connection
conn = engine.connect()

In [16]:
# check that all the info is there
Access.__table__

Table('access', MetaData(bind=None), Column('ip', String(), table=<access>, primary_key=True, nullable=False), Column('DateTime', String(), table=<access>), Column('Method', String(), table=<access>), Column('URL', String(), table=<access>), Column('ResponseCode', String(), table=<access>), Column('Referer', String(), table=<access>), Column('BytesSent', String(), table=<access>), Column('UserAgent', String(), table=<access>), schema=None)

In [17]:
Responses.__table__

Table('responses', MetaData(bind=None), Column('ip', String(), table=<responses>, primary_key=True, nullable=False), Column('latitude', Float(), table=<responses>), Column('longitude', Float(), table=<responses>), schema=None)

In [18]:
#delete tables to avoid duplicates when re-run
conn.execute(Access.__table__.delete())
conn.execute(Responses.__table__.delete())


In [19]:
# execute connection
conn.execute(Access.__table__.insert(), access)
conn.execute(Responses.__table__.insert(), responses)


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: access.ip [SQL: 'INSERT INTO access (ip, "DateTime", "Method", "URL", "ResponseCode", "Referer", "BytesSent", "UserAgent") VALUES (?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: (('62.170.2.205', '2018-04-08 07:19:25', 'HEAD', 'http://198.27.242.168:80/phpmyadmin/', '404', '-', '220', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'), ('62.170.2.205', '2018-04-08 07:19:25', 'HEAD', 'http://198.27.242.168:80/PMA/', '404', '-', '219', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'), ('62.170.2.205', '2018-04-08 07:19:25', 'HEAD', 'http://198.27.242.168:80/dbadmin/', '404', '-', '219', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'), ('62.170.2.205', '2018-04-08 07:19:26', 'HEAD', 'http://198.27.242.168:80/pma/', '404', '-'

In [ ]:
# check that it worked
engine.execute("select ip from access limit 10").fetchall()


In [ ]:
engine.execute('select ip from responses limit 10').fetchall()


In [ ]:
#check what's in there
engine.execute("select * from access limit 1").fetchall()


In [ ]:
engine.execute("select * from responses limit 1").fetchall()